In [2]:
import scipy.io.wavfile as wav
import numpy
import csv
import wave
import mod
from pymongo import *



def _wav2array(nchannels, sampwidth, data):
    """data must be the string containing the bytes from the wav file."""
    num_samples, remainder = divmod(len(data), sampwidth * nchannels)
    if remainder > 0:
        raise ValueError('The length of data is not a multiple of '
                         'sampwidth * num_channels.')
    if sampwidth > 4:
        raise ValueError("sampwidth must not be greater than 4.")

    if sampwidth == 3:
        a = numpy.empty((num_samples, nchannels, 4), dtype=numpy.uint8)
        raw_bytes = numpy.fromstring(data, dtype=numpy.uint8)
        a[:, :, :sampwidth] = raw_bytes.reshape(-1, nchannels, sampwidth)
        a[:, :, sampwidth:] = (a[:, :, sampwidth - 1:sampwidth] >> 7) * 255
        result = a.view('<i4').reshape(a.shape[:-1])
    else:
        # 8 bit samples are stored as unsigned ints; others as signed ints.
        dt_char = 'u' if sampwidth == 1 else 'i'
        a = numpy.fromstring(data, dtype='<%s%d' % (dt_char, sampwidth))
        result = a.reshape(-1, nchannels)
    return result


def cut_wave(wav_name, start_time, rate=44100, time_len=10):
    wav_reader = wave.open(wav_name)
    
    
    if(wav_reader.getnchannels() > 1):
        nchannels = wav_reader.getnchannels()
        sampwidth = wav_reader.getsampwidth()
        nframes = wav_reader.getnframes()
        data = wav_reader.readframes(nframes)
        wav_reader.close()
        rate = wav_reader.getframerate()
        signal = _wav2array(nchannels, sampwidth, data)
        mono = numpy.mean(signal, axis=1)
        
    else:
        (rate, signal) = wav.read(wav_name)
        mono = signal
        wav_reader.close()

    return mono[start_time*rate:(start_time+time_len)*rate]

    
def write_data(fname, family_name, wav_name, start_time):
    window = cut_wave(wav_name, start_time)
    csv_file = open("../wav_csv/"+fname+".csv", "a")    
    file_writer = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    
    sample_rate = 44100/64.0
    nsample = numpy.floor(len(window)/64.0)
    max_freq = sample_rate/2.0
    min_freq = (nsample/float(len(window)))
    mod_n = int(numpy.floor(nsample/2.0)+1)
    
    
    hz_spec = mod.hz_spec(window)
    hz_mod = mod.linear_mod(hz_spec, nsample)
    hz_logmod = mod.log_mod(hz_mod, mod_n)
    
    mean = numpy.mean(hz_logmod, axis=0)
    std = numpy.std(hz_logmod, axis=0)
    
    file_writer.writerow(numpy.append([family_name], hz_logmod.flatten()))
    
def write_short_data(fname, family_name, wav_name):
    csv_file= open("../wav_csv/"+fname+".csv", "a")
    
    wav_reader = wave.open(wav_name)
    rate = wav_reader.getframerate()
    nchannels = wav_reader.getnchannels()
    sampwidth = wav_reader.getsampwidth()
    nframes = wav_reader.getnframes()
    data = wav_reader.readframes(nframes)
    wav_reader.close()
    
    signal = _wav2array(nchannels, sampwidth, data)
    if(wav_reader.getnchannels() > 1):
        mono = numpy.mean(signal, axis=1)
    else:
        mono = signal
    
    if len(mono) < 44100*10:
        mono = numpy.append(mono, numpy.zeros(44100*10 - len(mono)))
        
        
    file_writer = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
    
    sample_rate = 44100/64.0
    nsample = numpy.floor(len(mono)/64.0)
    max_freq = sample_rate/2.0
    min_freq = (nsample/float(len(mono)))
    mod_n = int(numpy.floor(nsample/2.0)+1)
    
    
    hz_spec = mod.hz_spec(mono)
    hz_mod = mod.linear_mod(hz_spec, nsample)
    hz_logmod = mod.log_mod(hz_mod, mod_n)
    
    mean = numpy.mean(hz_logmod, axis=0)
    std = numpy.std(hz_logmod, axis=0)
    
    
    file_writer.writerow(numpy.append([family_name], hz_logmod.flatten()))
    
    
    
#######################################################################################################################################################################
#                                                 Cicada[98]
#######################################################################################################################################################################
def write_cicada(fname):
  
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5174_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5182_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5183_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5183_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5193_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5193_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5197_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5199_cicada.wav", 5)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5201_cicada.wav", 0)  
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5203_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5204_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5204_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5208_cicada.wav", 0)    
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5208_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5223_cicada.wav", 0)    
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5223_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5228_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5229_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5235_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5235_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5236_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5236_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5237_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5238_cicada.wav", 0)    
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5239_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5249_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5256_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5259_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5260_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5265_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5266_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5280_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5281_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5282_cicada.wav", 10)   
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5283_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5284_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5285_cicada.wav", 20) 
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5286_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5287_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5288_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5288_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5288_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5289_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5291_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5291_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5292_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5292_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5293_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5293_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5294_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5294_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5295_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5296_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5296_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5297_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5297_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5298_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5309_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5315_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5316_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5319_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5320_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5321_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5321_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5322_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5325_cicada.wav", 5)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5326_cicada.wav", 5)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5327_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5328_cicada.wav", 5)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5331_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5333_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5333_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5334_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5335_cicada.wav", 15)     
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5348_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5350_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5351_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5357_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5361_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5365_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5370_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5388_cicada.wav", 0) 
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5388_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5406_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5422_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5423_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5423_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5424_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5477_cicada.wav", 10)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5481_cicada.wav", 5)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5536_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5537_cicada.wav", 20)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5547_cicada.wav", 15)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5548_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5551_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5554_cicada.wav", 0)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5555_cicada.wav", 5)
    write_data(fname, "Cicadidae(cicada)", "../InsectData/CicadaHunt/5563_cicada.wav", 10)
    
    

#######################################################################################################################################################################
#                                                Tettigoniidae (bush-cricket)[112]
#######################################################################################################################################################################



##############[Phaneropterinae][11]################

#[sickle bearing bush-cricket][5]
def write_scickle_bearing_bush_cricket(fname):
    write_short_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/01 Sickle-bearing bush-cricket - call type 1 - 1 echeme - Baudewijn Ode.wav")
    write_short_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/01 Sickle-bearing bush-cricket - call type 1 - 1 echeme - example 2 - Baudewijn Ode.wav")
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/01 Sickle-bearing bush-cricket - call type 2 - series of syllables - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/01 Sickle-bearing bush-cricket - call type 2 - series of syllables - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/01 Sickle-bearing bush-cricket - call type 2 - series of syllables - Baudewijn Ode.wav", 20)

    
#[speckled bush-cricket][6]
def write_speckled_bush_cricket(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/03 Speckled bush-cricket - call of a few males - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/03 Speckled bush-cricket - call of a few males - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/03 Speckled bush-cricket - call of a male on the right, female replying on the left - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/03 Speckled bush-cricket - call of a male on the right, female replying on the left - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/03 Speckled bush-cricket - call of one male - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/03 Speckled bush-cricket - call of one male - Baudewijn Ode.wav", 0)


###############[Tettigoniidae] [17]###############

#[great green bush-cricket][17]
def write_great_green_bush_cricket(fname):

    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13015XXXXX-0028M0.WAV", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13015XXXXX-0028M0.WAV", 90)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13015XXXXX-0028M0.WAV", 140)  
    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000670 BD31.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000670 BD31.wav", 30) 
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000670 BD31.wav", 50)
    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000995 BD6.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000995 BD6.wav", 40)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000995 BD6.wav", 20)
    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001946 BD11.wav", 13)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001946 BD11.wav", 25)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001946 BD11.wav", 35)
        
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/08 Great green bush-cricket - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/08 Great green bush-cricket - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/08 Great green bush-cricket - at low temperature, others calling in the background - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/08 Great green bush-cricket - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/08 Great green bush-cricket - example 2 - Baudewijn Ode.wav", 10)


###############[Conocephalinae] [10]################

#[short winged conehead][5]
def write_short_winged_conehead(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/06 Short-winged conehead - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/06 Short-winged conehead - example 2 - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/06 Short-winged conehead - example 2 - Baudewijn Ode.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/06 Short-winged conehead - example 3 - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/06 Short-winged conehead - example 3 - Baudewijn Ode.wav", 10)

#[long winged conehead][5]
def write_long_winged_conehead(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000671 BD15.wav", 7)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/07 Long-winged conehead - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/07 Long-winged conehead - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/07 Long-winged conehead - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/07 Long-winged conehead - example 2 - Baudewijn Ode.wav", 10)



###############[Decticinae][74]################

#[dark bush-cricket][22]
def write_dark_bush_cricket(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13015XXXXX-0027M0.WAV", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13015XXXXX-0027M0.WAV", 20)
    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - 2 males alternating - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - 2 males alternating - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - at low temperature, Great green bush-cricket in the background - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - many echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - many echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - many echemes - Baudewijn Ode.wav", 17)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - many echemes - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/16 Dark bush-cricket - many echemes - example 2 - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/11089_dark.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/11089_dark.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8298_dark.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8298_dark.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8296_dark.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8296_dark.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8203_dark.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8203_dark.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8202_dark.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8202_dark.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8981_dark.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/8981_dark.wav", 20)
    
    
#[wartbiter][18]
def write_wartbiter(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD4.wav", 12)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD4.wav", 45)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD4.wav", 55)    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD4.wav", 80)
   
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD5.wav", 12)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD5.wav", 32)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD5.wav", 45)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD5.wav", 90)
    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD6.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD6.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001806 BD6.wav", 70)
    
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - example 2 - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - Baudewijn Ode.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - example 2 - Baudewijn Ode.wav", 15)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - example 2 - Baudewijn Ode.wav", 5)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/10 Wartbiter - Baudewijn Ode.wav", 15)

    
    
#[grey bush-cricket][6]
def write_grey_bush_cricket(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/12 Grey bush-cricket - many echemes - at low temperature - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/12 Grey bush-cricket - many echemes - at low temperature - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/12 Grey bush-cricket - many echemes - at low temperature - Baudewijn Ode.wav", 18)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/12 Grey bush-cricket - many echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/12 Grey bush-cricket - many echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/12 Grey bush-cricket - many echemes - Baudewijn Ode.wav", 18)
    
    
#[bog bush-cricket][6]
def write_bog_bush_cricket(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/13 Bog bush-cricket - many echemes - at low temperature - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/13 Bog bush-cricket - many echemes - at low temperature - Baudewijn Ode.wav", 8)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/13 Bog bush-cricket - many echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/13 Bog bush-cricket - many echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/13 Bog bush-cricket - many echemes - Baudewijn Ode.wav", 17)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/13 Bog bush-cricket - many echemes - example 2 - Baudewijn Ode.wav", 0)
    
    
#[roesel's bush-cricket][22]
def write_roesel_bush_cricket(fname):
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - at low temperature - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - at low temperature - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - at low temperature - Baudewijn Ode.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - long echeme - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - long echeme - Baudewijn Ode.wav", 10)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - short echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/UKorthoptera/15 Roesel's bush-cricket - short echemes - Baudewijn Ode.wav", 10)        
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/11103_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/11103_roesel.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/4308_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/4308_roesel.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7141_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7141_roesel.wav", 17)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7431_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7431_roesel.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7433_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7433_roesel.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7441_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7441_roesel.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7442_roesel.wav", 0)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7442_roesel.wav", 20)
    write_data(fname, "Tettigoniidae(bush cricket)", "../InsectData/CicadaHunt/7467_roesel.wav", 10)









#######################################################################################################################################################################
#                                                Gryllidae (cricket)[61]
#######################################################################################################################################################################


#################[Gryllinae][44]###################

#[field cricket][40]
def write_field_cricket(fname):
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0009-0010M0.WAV", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0009-0010M0.WAV", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0009-0010M0.WAV", 30)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0009-0010M0.WAV", 40)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0009-0010M0.WAV", 50)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0009-0010M0.WAV", 60)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0020-0036M0.WAV", 30)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0020-0036M0.WAV", 70)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0020-0036M0.WAV", 57)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 2/022A-WA08010X0020-0036M0.WAV", 90)
    
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0206M0.WAV", 10)      
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0206M0.WAV", 90)          
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0206M0.WAV", 60)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0207M0.WAV", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0207M0.WAV", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0207M0.WAV", 40)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0207M0.WAV", 30)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0208M0.WAV", 20)    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0208M0.WAV", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 4/022A-WA09025XXXXX-0208M0.WAV", 60)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA10012XXXXX-0599M0.WAV", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA10012XXXXX-0599M0.WAV", 30)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA10012XXXXX-0599M0.WAV", 60)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA10012XXXXX-0599M0.WAV", 70)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA10012XXXXX-0599M0.WAV", 90)

    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD26.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD26.wav", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD26.wav", 40)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD27.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD27.wav", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD27.wav", 30)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD27.wav", 40)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD28.wav", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0000996 BD28.wav", 10)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD4.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD4.wav", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD4.wav", 50)
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001207 BD2.wav", 8)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001207 BD2.wav", 20)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001207 BD2.wav", 30)
    
    
#[house cricket][17]
def write_house_cricket(fname):
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD8.WAV", 15)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD8.WAV", 25)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD8.WAV", 40)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD8.WAV", 50)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD8.WAV", 60)    
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 13)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 23)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 35)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 45)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 60)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 70)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 90)    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001969 BD9.WAV", 100)    
    
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/19 House cricket - many echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/19 House cricket - many echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/19 House cricket - several echemes - less active - Baudewijn Ode.wav", 0)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/19 House cricket - several echemes - less active - Baudewijn Ode.wav", 10)

    
    
    

#################[Nemobiinae][13]###################

#[wood cricket][13]
def write_wood_cricket(fname):
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD9.wav", 42)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD9.wav", 52)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD9.wav", 62)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD9.wav", 72)   
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD10.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD10.wav", 30)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/BritishLibrary/W1CDR files part 3/W1CDR0001017 BD10.wav", 58)
      
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/20 Wood cricket - chorus of many males - Baudewijn Ode.wav", 0)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/20 Wood cricket - long echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/20 Wood cricket - long echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/20 Wood cricket - short echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/20 Wood cricket - short echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/20 Wood cricket - short echemes - Baudewijn Ode.wav", 16)


#################[oecanthinae][1]####################

#[tree cricket][1]
def write_tree_cricket(fname):     
    write_data(fname, "Gryllidae(cricket)", "../InsectData/UKorthoptera/21 Tree cricket - various echemes - Baudewijn Ode.wav", 0)







#######################################################################################################################################################################
#                                                Acrididae (grasshopper)[79]
#######################################################################################################################################################################


####################[Gomphocerinae][75]#####################

#[field grasshopper][6]
def write_field_grasshopper(fname):
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/37 Field grasshopper - 1 series of echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/37 Field grasshopper - 1 series of echemes - Baudewijn Ode.wav", 8)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/37 Field grasshopper - 1 series of echemes - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/37 Field grasshopper - 1 series of echemes - example 2 - Baudewijn Ode.wav", 8)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/37 Field grasshopper - 4 echemes - at low temperature - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/37 Field grasshopper - 4 echemes - at low temperature - Baudewijn Ode.wav", 8)


#[lesser marsh grasshopper][2]
def write_lesser_marsh_grasshopper(fname):
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/40 Lesser marsh grasshopper - 1 series of echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/40 Lesser marsh grasshopper - 1 series of echemes - example 2 - Baudewijn Ode.wav", 0)



#[meadow grasshopper][27]
def write_meadow_grasshopper(fname):
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0259M0.WAV", 10)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0259M0.WAV", 20)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0259M0.WAV", 40) 
    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0266M0.WAV", 40)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0266M0.WAV", 20)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0266M0.WAV", 30)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0266M0.WAV", 10)
    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0268M0.WAV", 7)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0268M0.WAV", 20)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0268M0.WAV", 32)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0268M0.WAV", 54) 
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0268M0.WAV", 67)
    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0269M0.WAV", 7)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0269M0.WAV", 20)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0269M0.WAV", 30)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0269M0.WAV", 40)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0269M0.WAV", 50)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0269M0.WAV", 60)
    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0271M0.WAV", 7)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0271M0.WAV", 18)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0271M0.WAV", 28)  
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0271M0.WAV", 38) 


    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/42 Meadow grasshopper - at lower temperature - 2 echemes - other males in the background - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/42 Meadow grasshopper - at lower temperature - 2 echemes - other males in the background - Baudewijn Ode.wav", 9)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/42 Meadow grasshopper - several echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/42 Meadow grasshopper - several echemes - Baudewijn Ode.wav", 10)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/42 Meadow grasshopper - several echemes - Baudewijn Ode.wav", 17)

    
    
#[mottled grasshopper][8]
def write_motteled_grasshopper(fname):
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0264M0.WAV", 7)    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000636 BD23.WAV", 6)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000636 BD23.WAV", 16)    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000636 BD25.wav", 12)
    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/44 Mottled grasshopper - 1 series of echemes - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/44 Mottled grasshopper - 1 series of echemes - Baudewijn Ode.wav", 7)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/44 Mottled grasshopper - 1 series of echemes - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/44 Mottled grasshopper - 1 series of echemes - example 2 - Baudewijn Ode.wav", 7)


#[common green grasshopper][21]
def write_common_green_grasshopper(fname):
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0260M0.WAV", 30)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0260M0.WAV", 20)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0260M0.WAV", 60)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/022A files part 5/022A-WA13012XXXXX-0260M0.WAV", 44)
    
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000671 BD7.WAV", 10)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 1/W1CDR0000671 BD7.WAV", 20)
        
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 22)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 32)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 42)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 52)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 62)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 72)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 82)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 92)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 102)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 112)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/BritishLibrary/W1CDR files part 4/W1CDR0001964 BD20.WAV", 122)
        
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/31 Common green grasshopper - 1 echeme - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/31 Common green grasshopper - 1 echeme - Baudewijn Ode.wav", 10)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/31 Common green grasshopper - 1 echeme - example 2 - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/31 Common green grasshopper - 1 echeme - example 2 - Baudewijn Ode.wav", 10)


#[lesser mottled grasshopper][3]
#all recordings shorter than 10s
def write_lesser_motteled_grasshopper(fname):
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/30 Lesser Mottled Grasshopper - 1 echeme - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/30 Lesser Mottled Grasshopper - 1 echeme - example 2 - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/30 Lesser Mottled Grasshopper - 1 echeme - example 3 - Baudewijn Ode.wav")

    
    
#[woodland grasshopper][3]
#all recordings are shorter than 10s
def write_woodland_grasshopper(fname):
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/32 Woodland grasshopper - 1 echeme - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/32 Woodland grasshopper - 1 echeme - example 2 - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/32 Woodland grasshopper - 1 echeme - example 3 - Baudewijn Ode.wav")



#[heath grasshopper][3]
#all recordings are shorter than 10s
def write_heath_grasshopper(fname):
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/35 Heath grasshopper - 1 echeme - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/35 Heath grasshopper - 1 echeme - example 2 - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/35 Heath grasshopper - 1 echeme - example 3 - Baudewijn Ode.wav")

    
    
#[rufous grasshopper][2]
#all recordings are shorter than 10s
def write_rufous_grasshopper(fname):
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/45 Rufous grasshopper - 1 verse - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/45 Rufous grasshopper - 1 verse - example 2 - Baudewijn Ode.wav")



########################[Oedipodinae][4]######################

#[large marsh grasshopper][2]
#all recordings shorter than 10s
def write_large_marsh_grasshopper(fname):
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/26 Large marsh grasshopper - 1 series - Baudewijn Ode.wav")
    write_short_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/26 Large marsh grasshopper - 1 series - example 2 - Baudewijn Ode.wav")


#[strip_winged_grasshopper][2]
def write_strip_winged_grasshopper(fname):
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/29 Stripe-winged grasshopper - 1 echeme - Baudewijn Ode.wav", 0)
    write_data(fname, "Acrididae(grasshopper)", "../InsectData/UKorthoptera/29 Stripe-winged grasshopper - 1 echeme - example 2 - Baudewijn Ode.wav",0)


    
    
    
    
    
    
    
    
#######################################################################################################################################################################
#                                                Gryllotalpidae (mole cricket)[41]
#######################################################################################################################################################################
#[mole cricket][41]
def write_mole_cricket(fname):
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0012M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0012M0.WAV", 30)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0012M0.WAV", 40)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0012M0.WAV", 50)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0012M0.WAV", 60)
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 20)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 30)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 40)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 50)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 60)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 70)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0013M0.WAV", 80)
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0014M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0014M0.WAV", 20)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0014M0.WAV", 30)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0014M0.WAV", 40)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0014M0.WAV", 50)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0014M0.WAV", 60)

    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0015M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0015M0.WAV", 20)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0015M0.WAV", 30)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0015M0.WAV", 40) 
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 1/022A-WA06037X0063-0015M0.WAV", 50) 
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0012M0.WAV", 10) 
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0013M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0013M0.WAV", 20)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0013M0.WAV", 30)
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0014M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0014M0.WAV", 20)    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0014M0.WAV", 30)
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0016M0.wav", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0016M0.wav", 20)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0016M0.WAV", 30)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0016M0.wav", 50)
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0017M0.WAV", 10)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0017M0.WAV", 30)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0017M0.WAV", 40)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/BritishLibrary/022A files part 5/022A-WA12007X0001-0017M0.WAV", 60)
    
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/UKorthoptera/22 Mole cricket - Baudewijn Ode.wav", 0)
    write_data(fname, "Gryllotalpidae(mole cricket)", "../InsectData/UKorthoptera/22 Mole cricket - example 2 - Baudewijn Ode.wav", 0)    


    
    
    
#######################################################################################################################################################################
#                                                No Insect［98］
#######################################################################################################################################################################
def write_silent(fname):    
    
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3171_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3171_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3174_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3174_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3196_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3202_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3212_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3195_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3227_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3229_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3255_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3256_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3264_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3276_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3276_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3350_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3350_silent.wav", 20)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3992_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/3992_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4031_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4031_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4120_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4120_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4194_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4194_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4200_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4200_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4242_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4272_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4272_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4299_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4299_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4357_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4375_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4375_silent.wav", 15)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4377_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4420_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4428_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4242_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4390_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4391_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4395_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4420_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4428_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4490_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4504_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4506_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4514_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4514_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4516_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4635_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4786_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4788_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4842_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4892_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4952_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4954_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4959_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/4990_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5055_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5057_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5411_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5411_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5442_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5455_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5618_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5727_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5729_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5874_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5875_silent.wav", 5)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5877_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5900_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5957_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5914_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/5914_silent.wav", 20)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/6143_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/6386_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/6563_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/6736_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7031_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7108_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7582_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7635_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7804_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7457_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/7810_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/9828_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/8099_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/8235_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/8294_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/8310_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/8342_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/8347_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/9826_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/9918_silent.wav", 0)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/10933_silent.wav", 10)
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/10976_silent.wav", 0)   
    write_data(fname, "No Insect", "../InsectData/CicadaHunt/11129_silent.wav", 0)




    
csv_file = open("../wav_csv/insects.csv", "w")    
file_writer = csv.writer(csv_file, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL)
labels = numpy.append(["label"], ["mod%s" % i for i in range(33*48)])
file_writer.writerow(labels)

write_cicada("insects")

write_scickle_bearing_bush_cricket("insects")
write_speckled_bush_cricket("insects")
write_great_green_bush_cricket("insects")
write_short_winged_conehead("insects")
write_long_winged_conehead("insects")
write_dark_bush_cricket("insects")
write_wartbiter("insects")
write_grey_bush_cricket("insects")
write_bog_bush_cricket("insects")
write_roesel_bush_cricket("insects")

write_field_cricket("insects")
write_house_cricket("insects")
write_wood_cricket("insects")
write_tree_cricket("insects")

write_field_grasshopper("insects")
write_lesser_marsh_grasshopper("insects")
write_meadow_grasshopper("insects")
write_motteled_grasshopper("insects")
write_common_green_grasshopper("insects")
write_lesser_motteled_grasshopper("insects")
write_woodland_grasshopper("insects")
write_heath_grasshopper("insects")
write_rufous_grasshopper("insects")
write_large_marsh_grasshopper("insects")
write_strip_winged_grasshopper("insects")

write_mole_cricket("insects")

write_silent("insects")
print("done")



done


/Library/Frameworks/Python.framework/Versions/3.4/lib/python3.4/site-packages/scipy/io/wavfile.py:172: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)
